#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import streamlit as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy


In [2]:
disease = "Malaria"
d = "Allergy"
drugs = "PCM"
options = "Skin_rash"

In [ ]:
# Create variables
real_disease = disease
predicted_disease = d
drugs_on = drugs
user_symptoms = options

# Make into a dictionary
dic = {"symptoms": user_symptoms,"predicted_disease" :predicted_disease,  "disease": real_disease, "drugs_on": drugs}

# Create Dataframe
data = pd.DataFrame(dic, index=range(1))

# Concatenate
#updated_data = pd.concat([])

In [21]:
disease = "Acne"
d = "Fungal infection"
drugs = "Benzoic Peroxide"
options = "Skin_rash", "itching"

In [25]:
dic = {"symptoms": options,"predicted_disease" :d,  "disease": disease, "drugs_on": drugs}

updated_data = pd.DataFrame(dic, index=range(len(options)))
updated_data.to_csv("updated_data.csv")


In [16]:
dic_g = {"symptoms": options,"predicted_disease" :d,  "disease": disease, "drugs_on": drugs}
g = pd.DataFrame(dic_g, index=range(len(options)))
g

,symptoms,predicted_disease,disease,drugs_on
0,Skin_rash,Allergy,Acne,Benzoic Peroxide
1,itching,Allergy,Acne,Benzoic Peroxide


In [20]:
pd.concat([f,g])

,symptoms,predicted_disease,disease,drugs_on
0,Skin_rash,Allergy,Malaria,PCM
0,Skin_rash,Allergy,Acne,Benzoic Peroxide
1,itching,Allergy,Acne,Benzoic Peroxide


#### Load Datasets

In [19]:
print(tf.__version__)

2.10.0


In [15]:
df_1 = pd.read_csv("dataset.csv")

print("Dataset shape is:", df_1.shape)

df_1.head()

Dataset shape is: (4925, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acne,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Acne,skin_rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Acne,skin_rash,pus_filled_pimples,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
def uniq_symp(df):
    a = df["Symptom_1"].unique().tolist()
    b = df["Symptom_2"].unique().tolist()
    c = df["Symptom_3"].unique().tolist()
    d = df["Symptom_4"].unique().tolist()
    e = df["Symptom_5"].unique().tolist()
    f = df["Symptom_6"].unique().tolist()
    g = df["Symptom_7"].unique().tolist()
    h = df["Symptom_8"].unique().tolist()
    i = df["Symptom_9"].unique().tolist()
    j = df["Symptom_10"].unique().tolist()
    k = df["Symptom_11"].unique().tolist()
    l = df["Symptom_12"].unique().tolist()
    m= df["Symptom_13"].unique().tolist()
    n = df["Symptom_14"].unique().tolist()
    o = df["Symptom_15"].unique().tolist()
    p = df["Symptom_16"].unique().tolist()
    q= df["Symptom_17"].unique().tolist()



    z = a+b+c+d+e+f+g+h+i+j+k+l+m+o+p+q
    symp = {"Symptoms": z}
    df = pd.DataFrame(symp, index=range(len(z)))
    uniq = df["Symptoms"].unique().tolist()

    return uniq

In [2]:
df_2 = pd.read_csv("symptom_Description.csv")

print("Description shape is:", df_2.shape)

df_2.head()

Description shape is: (42, 2)


,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [4]:
df_3 = pd.read_csv("symptom_precaution.csv")

print("Precaution shape is:", df_3.shape)
#df_3.fillna()
df_3.head()

Precaution shape is: (41, 5)


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug_Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,use ice to compress itching,know and avoid allergens
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [3]:
"""A function that takes in a disease name , gives you a discription about the disease and precautions to take."""
def drug_info(disease):
    d = disease

    # Print disease description
    print("The predicted disease is:", d.strip().replace("_", " "))
    print(" ")
    print("Description of", d.strip().replace("_", " "),":")
    print(df_2[df_2["Disease"] == d]["Description"].item())
    print(" ")

    # Print Precautions to take
    print("Precautions to take are:")
    print(df_3[df_3["Disease"] == d]["Precaution_1"].item().title()) 
    print(df_3[df_3["Disease"] == d]["Precaution_2"].item().title())
    print(df_3[df_3["Disease"] == d]["Precaution_3"].item().title())
    print(df_3[df_3["Disease"] == d]["Precaution_4"].item().title())

In [14]:
drug_info("Malaria")

The predicted disease is: Malaria
 
Description of Malaria :
An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
 
Precautions to take are:
Consult Nearest Hospital
Avoid Oily Food
Avoid Non Veg Food
Keep Mosquitos Out


In [18]:
df_4 = pd.read_csv("Symptom-severity.csv")

print("Severity shape is:", df_4.shape)

df_4.head()

Severity shape is: (133, 2)


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [25]:
def encode_symptoms(df, df_4):
    for i in df_4.index:
        symptom = df_4["Symptom"][i]
        weight = df_4["weight"][i]
        df = df.replace(symptom, weight)

    # Replace missing values with 0
    df = df.fillna(0)

    # Additional hardcoded replacements
    df = df.replace("foul_smell_of_urine", 5)
    df = df.replace("dischromic__patches", 6)
    df = df.replace("spotting__urination", 6)
    
    return df

In [26]:
new_df_1 = encode_symptoms(df_1, df_4)
new_df_1.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,3,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acne,3,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acne,2,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acne,3,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acne,3,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_df = pd.read_csv("Training.csv")
test_df = pd.read_csv("Testing.csv")

In [31]:
list(train_df.columns)

['itching',
 'skin_rash',
 'nodal_skin_eruptions',
 'continuous_sneezing',
 'shivering',
 'chills',
 'joint_pain',
 'stomach_pain',
 'acidity',
 'ulcers_on_tongue',
 'muscle_wasting',
 'vomiting',
 'burning_micturition',
 'spotting_ urination',
 'fatigue',
 'weight_gain',
 'anxiety',
 'cold_hands_and_feets',
 'mood_swings',
 'weight_loss',
 'restlessness',
 'lethargy',
 'patches_in_throat',
 'irregular_sugar_level',
 'cough',
 'high_fever',
 'sunken_eyes',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'yellowish_skin',
 'dark_urine',
 'nausea',
 'loss_of_appetite',
 'pain_behind_the_eyes',
 'back_pain',
 'constipation',
 'abdominal_pain',
 'diarrhoea',
 'mild_fever',
 'yellow_urine',
 'yellowing_of_eyes',
 'acute_liver_failure',
 'fluid_overload',
 'swelling_of_stomach',
 'swelled_lymph_nodes',
 'malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',


## Dropping the least significant parameters from the dataset

In [33]:
y_train=train_df['prognosis']
x_train=train_df.drop(['sinus_pressure',
 'irritation_in_anus',
 'swollen_blood_vessels',
 'enlarged_thyroid',
 'brittle_nails',
 'drying_and_tingling_lips',
 'slurred_speech',
 'spinning_movements',
 'loss_of_balance',
 'continuous_feel_of_urine',
 'internal_itching',
 'toxic_look_(typhos)',
 'altered_sensorium',
 'dischromic _patches',
 'family_history',
 'mucoid_sputum',
 'rusty_sputum',
 'receiving_unsterile_injections',
 'coma',
 'stomach_bleeding',
 'distention_of_abdomen',
 'fluid_overload.1',
 'prominent_veins_on_calf',
 'silver_like_dusting',
 'small_dents_in_nails',
 'inflammatory_nails',
 'yellow_crust_ooze',
 'prognosis'],axis=1)


In [34]:
x_test=test_df.drop([
 'sinus_pressure',
 'irritation_in_anus',
 'swollen_blood_vessels',
 'enlarged_thyroid',
 'brittle_nails',
 'drying_and_tingling_lips',
 'slurred_speech',
 'spinning_movements',
 'loss_of_balance',
 'continuous_feel_of_urine',
 'internal_itching',
 'toxic_look_(typhos)',
 'altered_sensorium',
 'dischromic _patches',
 'family_history',
 'mucoid_sputum',
 'rusty_sputum',
 'receiving_unsterile_injections',
 'coma',
 'stomach_bleeding',
 'distention_of_abdomen',
 'fluid_overload.1',
 'prominent_veins_on_calf',
 'silver_like_dusting',
 'small_dents_in_nails',
 'inflammatory_nails',
 'yellow_crust_ooze',
 'prognosis'],axis=1)
y_test=test_df['prognosis']

In [35]:
x_train.columns

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue',
       ...
       'history_of_alcohol_consumption', 'blood_in_sputum', 'palpitations',
       'painful_walking', 'pus_filled_pimples', 'blackheads', 'scurring',
       'skin_peeling', 'blister', 'red_sore_around_nose'],
      dtype='object', length=105)

### Data Standardization

In [36]:
le=LabelEncoder()
dummy=le.fit_transform(train_df['prognosis'])
y_train=pd.DataFrame(dummy)
y_train.head()

,0
0,15
1,15
2,15
3,15
4,15


## Listing the indices of diseases

In [37]:
disease_classes = le.classes_ 
disease_classes

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes ',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'No disease', 'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [38]:
dummy=LabelEncoder().fit_transform(test_df['prognosis'])
y_test=pd.DataFrame(dummy)

In [39]:
model=Sequential([
    Dense(units=200,activation='relu'),
    Dense(units=150,activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=42,activation='softmax')
])

In [44]:
y_test.head()

,0
0,15
1,4
2,16
3,9
4,14


In [45]:
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])
# model.summary

In [46]:
history=model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test))

Epoch 1/10


c:\Users\ThisPC\anaconda3\envs\tf-new\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


39/39 [==============================] - 2s 12ms/step - loss: 2.6816 - accuracy: 0.6259 - val_loss: 1.0702 - val_accuracy: 0.9070
Epoch 2/10
 1/39 [..............................] - ETA: 0s - loss: 0.9912 - accuracy: 0.9375

c:\Users\ThisPC\anaconda3\envs\tf-new\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


39/39 [==============================] - 0s 8ms/step - loss: 0.2988 - accuracy: 0.9907 - val_loss: 0.1634 - val_accuracy: 0.9535
Epoch 3/10
39/39 [==============================] - 0s 6ms/step - loss: 0.0202 - accuracy: 0.9990 - val_loss: 0.1535 - val_accuracy: 0.9535
Epoch 4/10
39/39 [==============================] - 0s 7ms/step - loss: 0.0103 - accuracy: 0.9994 - val_loss: 0.1475 - val_accuracy: 0.9535
Epoch 5/10
39/39 [==============================] - 0s 6ms/step - loss: 0.0066 - accuracy: 0.9998 - val_loss: 0.1409 - val_accuracy: 0.9535
Epoch 6/10
39/39 [==============================] - 0s 6ms/step - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.1348 - val_accuracy: 0.9535
Epoch 7/10
39/39 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 0.9998 - val_loss: 0.1278 - val_accuracy: 0.9535
Epoch 8/10
39/39 [==============================] - 0s 6ms/step - loss: 0.0031 - accuracy: 0.9998 - val_loss: 0.1220 - val_accuracy: 0.9535
Epoch 9/10
39/39 [=============

In [47]:
prediction=model.predict(x_test)
prediction[1]

2/2 [==============================] - 0s 3ms/step


array([8.6895725e-06, 1.2352216e-06, 7.9257836e-09, 2.6227712e-08,
       9.9985147e-01, 1.3794875e-07, 3.8764043e-08, 8.2187812e-09,
       1.1220064e-07, 1.1415300e-07, 6.2627514e-06, 1.6098907e-07,
       7.9085387e-09, 4.0167783e-06, 3.5286141e-10, 7.1084401e-06,
       2.7502505e-08, 1.1166558e-06, 8.2157139e-06, 6.7725651e-08,
       6.4588814e-05, 2.0289901e-09, 6.8305241e-09, 2.0363585e-05,
       4.7632316e-08, 6.2072938e-09, 8.9455689e-08, 5.0784379e-08,
       2.0110789e-08, 1.7564923e-07, 8.6878691e-09, 7.0298665e-06,
       1.3223720e-07, 4.8027705e-06, 6.8492362e-08, 1.0323178e-05,
       1.0126846e-06, 5.9953356e-07, 1.0729898e-06, 1.6672470e-07,
       5.0927531e-07, 3.0488987e-08], dtype=float32)

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'], label='train')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.show()

# plt.plot(history.history['accuracy'])
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.show()


In [ ]:
pred=np.argmax(prediction, axis=1)
original=y_test[0]
print(original[41])

In [ ]:
# confusion = confusion_matrix(original,pred)

# fig = px.imshow(confusion, labels=dict(x="Predicted Value", y="Actual Vlaue"),text_auto=True, title='Confusion Matrix')
# fig.update_layout(title_x=0.5)
# fig.show()

In [49]:
# count=0
# for i in range(41):
#     if(pred[i]-original[i]!=0):
#         count+=1

# Test_accuracy=(42-count)/42*100
# for i in range(10):
#     print(f"Predicted: {pred[i]}, Actual: {original[i]}")

In [ ]:
pred_ga=np.argmax(ga, axis=1)
pred_ga[0]

####  Random Forest Classifier

In [50]:
clf = RandomForestClassifier(n_estimators=100, random_state=42) 
clf.fit(x_train, np.ravel(y_train))

RandomForestClassifier(random_state=42)

In [51]:
rf_predictions = clf.predict(x_test)
print(len(rf_predictions))

43


In [52]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, rf_predictions)
mse = mean_squared_error(y_test, rf_predictions)
r2 = r2_score(y_test, rf_predictions)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 0.4883720930232558
Mean Squared Error: 10.255813953488373
R-squared: 0.9288715013390747


In [54]:
clf.score(x_test, y_test)

0.9767441860465116

In [55]:
#print(f"Random Forest Test Accuracy:",Test_accuracy,'%')

In [56]:
# count=0
# for i in range(41):
#     if(rf_predictions[i]-original[i]!=0):
#         count+=1

# rf_Test_accuracy=(42-count)/42*100

## Saving the Model

In [57]:
from joblib import dump,load

In [58]:
filename = 'rf_model.joblib'
model = clf
dump(model, filename)

['rf_model.joblib']

In [59]:
ml_model=load('rf_model.joblib')

In [63]:
inp = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

# Reshape the 1D array into a 2D array with 1 row and 48 columns
inp = inp.reshape(1,-1)

# Make the prediction
prediction = ml_model.predict(inp)
print(prediction)

[31]


C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [64]:
disease_classes[31]

'No disease'

In [ ]:
da_list = da.iloc[0].to_list()
da_arr = np.array(da_list)
da_arr = da_arr.reshape(1,-1)

# Make the prediction
prediction = ml_model.predict(da_arr)
prediction
dis = prediction[0]
disease_classes[dis]

### Initial Random Forest based on encoding disease severity

In [ ]:
def encode_symptoms(df, data_severity):
    for i in data_severity.index:
        symptom = data_severity["Symptom"][i]
        weight = data_severity["weight"][i]
        df = df.replace(symptom, weight)

    # Replace missing values with 0
    df = df.fillna(0)

    # Additional hardcoded replacements
    df = df.replace("foul_smell_of_urine", 5)
    df = df.replace("dischromic__patches", 6)
    df = df.replace("spotting__urination", 6)
    
    return df

In [ ]:
new_df = encode_symptoms(df, data_severity)

In [16]:
# separating the data and labels
X = new_df.drop(columns='Disease', axis=1)
Y = new_df['Disease']

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X)

In [ ]:
X = scaler.transform(X)

In [133]:
from sklearn.ensemble import RandomForestClassifier
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rfc_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rfc_classifier.fit(X_train, Y_train)

# Predict disease labels on the testing data
Y_pred = rfc_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)

precision = precision_score(Y_test, Y_pred, average='macro')
recall = recall_score(Y_test, Y_pred,average='macro')
f1 = f1_score(Y_test, Y_pred, average='macro')

print("Random Forest")
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(accuracy_score(Y_test, Y_pred,normalize=False))
print("Confusion matrix")
conf_matrix=confusion_matrix(Y_test,Y_pred)
print(conf_matrix)

Random Forest
Accuracy: 0.9908536585365854
Precision: 0.9916531165311654
Recall: 0.9918172648227722
F1-Score: 0.9914105369307706
975
Confusion matrix
[[31  0  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 22  0]
 [ 0  0  0 ...  0  0 24]]


In [77]:
"""
A function that takes in a list of numbers corresponsing to the symptom. Returns a dataframe suitable to undergo prediction
"""
def dis_data(values):
    symp_keys = x_train.columns
    symp_values = values
    dic = {symp_keys[i]: symp_values[i] for i in range(len(symp_keys))}
    df = pd.DataFrame(list(dic.items())).transpose()
    df.columns = df.iloc[0]
    df.drop(index=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns="index", inplace=True)

    return df

In [88]:
f = pd.read_csv("empty_df.csv")
f.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,history_of_alcohol_consumption,blood_in_sputum,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,blister,red_sore_around_nose
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
f.columns

Index(['Unnamed: 0', 'itching', 'skin_rash', 'nodal_skin_eruptions',
       'continuous_sneezing', 'shivering', 'chills', 'joint_pain',
       'stomach_pain', 'acidity',
       ...
       'history_of_alcohol_consumption', 'blood_in_sputum', 'palpitations',
       'painful_walking', 'pus_filled_pimples', 'blackheads', 'scurring',
       'skin_peeling', 'blister', 'red_sore_around_nose'],
      dtype='object', length=106)

In [75]:
d = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#d = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
da = dis_data(d)
#da["itching"] = 1
#g = da.iloc[0].to_list()
ga = np.asarray(da).astype(np.float32)
model.predict(ga)
da.to_csv("empty_df.csv")

KeyError: "['Unnamed:0'] not found in axis"